In [97]:
from datetime import datetime
import os
import shutil

fortnite_video_directory_path = "test_videos/"
manual_date_time_file_path = fortnite_video_directory_path + "/date_list.txt"
selected_files_file_path = fortnite_video_directory_path + "/selected_files_list.txt"
upload_directory_path = fortnite_video_directory_path + "/uploads/"

fortnite_clip_search_limit_in_seconds = 60

manual_date_time_format = "%y.%m.%d - %H.%M.%S"
fortnite_highlight_date_time_format = "%Y.%m.%d - %H.%M.%S"

In [98]:
# List the directories in the Fortnite folder && date_list file
all_files = os.listdir(fortnite_video_directory_path)
fortnite_highlight_files = [files.strip() for files in all_files if files.rfind("DVR") >= 0]
print(f"fortnite_files: {fortnite_highlight_files}")
with open(manual_date_time_file_path) as file:
    manual_date_times = [datetime.strptime(line.strip(), manual_date_time_format) for line in file if line]

print(f"fortnite_game_date_times: {fortnite_game_date_times}")

fortnite_files: ['Fortnite 2022.10.05 - 21.37.30.34.DVR.mp4', 'Fortnite 2022.10.05 - 21.35.31.33.DVR.mp4']
fortnite_game_date_times: [datetime.datetime(2022, 10, 5, 21, 37, 33)]


In [99]:
# Convert to date/time && map times to filenames
fortnite_highlight_date_to_file_map = \
    dict([ (datetime.strptime( \
        fortnite_file.replace(".DVR", "").replace("Fortnite ", "").replace(".mp4","")[:-3], \
        fortnite_highlight_date_time_format), fortnite_file) for fortnite_file in fortnite_files ])
print(f"fortnite_dates: {fortnite_highlight_date_to_file_map}")


fortnite_dates: {datetime.datetime(2022, 10, 5, 21, 37, 30): 'Fortnite 2022.10.05 - 21.37.30.34.DVR.mp4', datetime.datetime(2022, 10, 5, 21, 35, 31): 'Fortnite 2022.10.05 - 21.35.31.33.DVR.mp4'}


In [104]:
# TODO: Get files that have "close" times to date_list file
files_chosen_for_fortnite_game_times = {} #{manual_date:fortnite_date_list}
for manual_date_time in manual_date_times:
    for fortnite_file_date_time, fortnite_file in fortnite_highlight_date_to_file_map.items():
        absolute_time_diff_in_seconds = abs((manual_date_time - fortnite_file_date_time).total_seconds())
        print(f"time_diff: {absolute_time_diff_in_seconds}")
        if 0 < absolute_time_diff_in_seconds < fortnite_clip_search_limit_in_seconds:
            print(f"FOUND A CLIP: {fortnite_file}")
            if manual_date_time in files_chosen_for_fortnite_game_times:
                files_chosen_for_fortnite_game_times[manual_date_time].append(fortnite_file)
            else:
                files_chosen_for_fortnite_game_times[manual_date_time] = [fortnite_file]

print(f"files_chosen: {files_chosen_for_fortnite_game_times}")  

# write all clips found to file
with open(selected_files_file_path, 'w') as selected_files_file:
    for (date, files) in files_chosen_for_fortnite_game_times.items():
        #selected_files_file.write('\n'.join(files))
        selected_files_file.write(str(date) + '\n')
        for file in files:
            selected_files_file.write('\t' + file)
            shutil.copyfile(fortnite_video_directory_path + file, upload_directory_path + file) 
            

time_diff: 117.0
time_diff: 2.0
FOUND A CLIP: Fortnite 2022.10.05 - 21.35.31.33.DVR.mp4
files_chosen: {datetime.datetime(2022, 10, 5, 21, 35, 33): ['Fortnite 2022.10.05 - 21.35.31.33.DVR.mp4']}
